# Getting Started: Building Agents over LlamaCloud RAG Pipelines

<a href="https://colab.research.google.com/github/run-llama/llamacloud-demo/blob/main/examples/getting_started_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook we show you how to build a function calling agent (powered by OpenAI) over RAG pipelines built with LlamaCloud.

Adding an agentic layer to RAG allows you to build in a layer of query planning and state management that allows you to ask multi-part complex questions over existing RAG pipelines and get back answers in a conversational manner.


## Initial Setup 

Let's start by importing some simple building blocks.  

The main thing we need is:
1. the OpenAI API (using our own `llama_index` LLM class)
2. a place to keep conversation history 
3. a definition for tools that our agent can use.

If you're opening this Notebook on colab, you will probably need to install LlamaIndex 🦙.


In [ ]:
%pip install llama-index-indices-llama-cloud
%pip install llama-index

In [1]:
import nest_asyncio
nest_asyncio.apply()

from IPython.display import Markdown, display

Make sure your OPENAI_API_KEY is set. Otherwise explicitly specify the `api_key` parameter.

In [2]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-4")

## Build Two LlamaCloud Indexes

Our data sources are the 2021 Lyft and Uber 10K documents.

In contrast to the other getting started examples, in this notebook we will build **two** RAG pipelines: one for Uber and one for Lyft. This is for the sake of example; we can plug in both RAG pipelines as tools for the agent to reason over.

To create each index, follow the instructions:
1. You can download them here ([Uber 10K](https://www.dropbox.com/s/te0a2w227v27iag/uber_2021.pdf?dl=1), [Lyft 10K](https://www.dropbox.com/s/qctkz6nxhm0y5qe/lyft_2021.pdf?dl=1))
2. Follow instructions on `https://cloud.llamaindex.ai/` to signup for an account. Create a pipeline by uploading these documents.

In [3]:
from llama_index.indices.managed.llama_cloud import LlamaCloudIndex

index_uber = LlamaCloudIndex(
  name="<index_uber>", 
  project_name="<project_name>",
  api_key="llx-"
)
index_lyft = LlamaCloudIndex(
  name="<index_lyft>", 
  project_name="<project_name>",
  api_key="llx-"
)

For each index, get a query engine from the index, which gives us an out-of-the-box RAG pipeline.

In [5]:
query_engine_uber = index_uber.as_query_engine(llm=llm)
query_engine_lyft = index_lyft.as_query_engine(llm=llm)

## OpenAI Agent over LlamaCloud RAG Pipelines

We convert both query engines to tools and pass it to a function calling OpenAI agent.

In [6]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

tool_uber = QueryEngineTool(
    query_engine=query_engine_uber,
    metadata=ToolMetadata(
        name="uber_10k",
        description=(
            "Provides information about Uber financials for year 2021. "
            "Use a detailed plain text question as input to the tool."
        ),
    ),
)
tool_lyft = QueryEngineTool(
    query_engine=query_engine_lyft,
    metadata=ToolMetadata(
        name="lyft_10k",
        description=(
            "Provides information about Lyft financials for year 2021. "
            "Use a detailed plain text question as input to the tool."
        ),
    ),
)

In [7]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

agent_worker = FunctionCallingAgentWorker.from_tools(
    [tool_uber, tool_lyft], llm=llm, verbose=True
)
agent = AgentRunner(agent_worker)

In [8]:
response = agent.chat("Tell me both the tailwinds for Uber and Lyft?")

Added user message to memory: Tell me both the tailwinds for Uber and Lyft?
=== Calling Function ===
Calling function: uber_10k with args: {"input": "What were the tailwinds for Uber in 2021?"}
=== Function Output ===
In 2021, Uber experienced several positive factors. The company's revenue increased by $6.3 billion, or 57%, primarily due to a 56% increase in Gross Bookings. This was largely driven by a 71% increase in Delivery Gross Bookings, as a result of increased food delivery orders and higher basket sizes due to stay-at-home orders related to COVID-19. The company also saw growth in Mobility Gross Bookings by 38%, due to increases in Trip volumes as the business recovers from the impacts of COVID-19. Additionally, there was an increase in Delivery revenue resulting from an increase in certain Courier payments and incentives. The growth of Delivery was accelerated in 2021 due to COVID-19 related restrictions. The company also launched new services or expanded existing ones, parti

In [9]:
display(Markdown(f"{response}"))

In 2021, both Uber and Lyft experienced several tailwinds:

For Uber:
1. The company's revenue increased by $6.3 billion, or 57%, primarily due to a 56% increase in Gross Bookings. 
2. There was a 71% increase in Delivery Gross Bookings, as a result of increased food delivery orders and higher basket sizes due to stay-at-home orders related to COVID-19. 
3. Uber saw growth in Mobility Gross Bookings by 38%, due to increases in Trip volumes as the business recovers from the impacts of COVID-19. 
4. There was an increase in Delivery revenue resulting from an increase in certain Courier payments and incentives. 
5. The growth of Delivery was accelerated in 2021 due to COVID-19 related restrictions. 
6. The company also launched new services or expanded existing ones, particularly those related to food and goods delivery, in response to the pandemic.

For Lyft:
1. The distribution of vaccines and reopening of communities led to a recovery in demand for their platform. 
2. This resulted in a 36% increase in revenue compared to the previous year and a 49.2% increase in the number of Active Riders in the fourth quarter of 2021 compared to the same period in 2020. 
3. The company saw a decrease in net loss by $743.5 million, or 42%, from 2020. 
4. Lyft completed a transaction with Woven Planet, a subsidiary of Toyota Motor Corporation, which resulted in a pre-tax gain of $119.3 million. 
5. The company also achieved its first annual Adjusted EBITDA profitability in 2021.

In [10]:
response = agent.chat("What are the investments made by Uber?")

Added user message to memory: What are the investments made by Uber?
=== Calling Function ===
Calling function: uber_10k with args: {"input": "What were the investments made by Uber in 2021?"}
=== Function Output ===
In 2021, Uber made strategic investments in Grab and Aurora, which became publicly listed within the year. These investments have been classified as marketable equity securities with a readily determinable fair value. Uber also continued to invest in new platform offerings to strengthen their platform and existing offerings.
=== LLM Response ===
In 2021, Uber made the following investments:

1. Strategic investments in Grab and Aurora, which became publicly listed within the year. These investments have been classified as marketable equity securities with a readily determinable fair value.
2. The company also continued to invest in new platform offerings to strengthen their platform and existing offerings.


In [11]:
display(Markdown(f"{response}"))

In 2021, Uber made the following investments:

1. Strategic investments in Grab and Aurora, which became publicly listed within the year. These investments have been classified as marketable equity securities with a readily determinable fair value.
2. The company also continued to invest in new platform offerings to strengthen their platform and existing offerings.

In [12]:
response = agent.chat("Compare the investments made by Uber and lyft?")

Added user message to memory: Compare the investments made by Uber and lyft?
=== Calling Function ===
Calling function: lyft_10k with args: {"input": "What were the investments made by Lyft in 2021?"}
=== Function Output ===
In 2021, Lyft made several investments. They continued to invest in their mission, including research and development, strategic acquisitions, sales, and marketing. They also invested in the expansion of their network of Light Vehicles and Lyft Autonomous, focusing on the deployment and scaling of third-party self-driving technology. They made payments in excess of $300 million under an amended arrangement with AWS. They also invested in the bikeshare program for the New York metro area, with investments totaling $87.1 million as of December 31, 2021. Additionally, they made payments totaling $23.1 million and investments totaling $23.5 million as part of a noncancelable arrangement with the City of Chicago for the Divvy bike share program. They also continued to i

In [13]:
display(Markdown(f"{response}"))

In 2021, both Uber and Lyft made several investments:

For Uber:
1. Uber made strategic investments in Grab and Aurora, which became publicly listed within the year. These investments have been classified as marketable equity securities with a readily determinable fair value.
2. The company also continued to invest in new platform offerings to strengthen their platform and existing offerings.

For Lyft:
1. Lyft continued to invest in their mission, including research and development, strategic acquisitions, sales, and marketing.
2. They invested in the expansion of their network of Light Vehicles and Lyft Autonomous, focusing on the deployment and scaling of third-party self-driving technology.
3. They made payments in excess of $300 million under an amended arrangement with AWS.
4. Lyft invested in the bikeshare program for the New York metro area, with investments totaling $87.1 million as of December 31, 2021.
5. They made payments totaling $23.1 million and investments totaling $23.5 million as part of a noncancelable arrangement with the City of Chicago for the Divvy bike share program.
6. They also continued to invest in their data privacy and security foundation and their intellectual property, holding 343 issued U.S. patents and 310 U.S. patent applications pending as of the end of 2021.